In [1]:
import pandas as pd
import os
import math
import scipy.stats as st
import scipy.spatial.distance as sd
import sklearn.metrics as mt
import numpy as np


### Load 'procedure' dataset

In [2]:
path='data-all/data-procedure'
subjects_proc = os.listdir(path)
subjects_proc =[x for x in subjects_proc if x.endswith(".csv")]

df_procedure=pd.DataFrame()

for proc in subjects_proc:
    sub_name=proc.split('.csv')[0]
    df = pd.read_csv("{}/{}.csv".format(path,sub_name),delimiter=';',low_memory = False)
    
    sub_name=sub_name.split('-')[0]
    df['SUB-NAME'] = df.apply(lambda row: sub_name, axis=1)
    df = df[['SUB-NAME','TIMESTAMP','ID','COND','IADS-ID','IAPS-ID','ANS-VALENCE','ANS-AROUSAL','ANS-TIME','EVENT']]
    frames=[df_procedure,df]
    df_procedure=pd.concat(frames)
    
affective_condition_lst=['P+','P-','S+','S-','PS+','PS-','P0','S0','PS0']

df_procedure= df_procedure[df_procedure['COND'].isin(affective_condition_lst)]

df_procedure=df_procedure.reset_index()
df_procedure['STIMULI'] = df_procedure.apply(lambda row:'_'.join([str(df_procedure.loc[row.name,'SUB-NAME']),str(df_procedure.loc[row.name,'IADS-ID']),str(df_procedure.loc[row.name,'IAPS-ID'])]) , axis=1)
del df_procedure['index']

In [3]:
df_procedure

,SUB-NAME,TIMESTAMP,ID,COND,IADS-ID,IAPS-ID,ANS-VALENCE,ANS-AROUSAL,ANS-TIME,EVENT,STIMULI
0,SUB572,1.580992e+09,572.0,PS-,276,6360,3.768,5.669,4.817,NaN,SUB572_276_6360
1,SUB572,1.580992e+09,572.0,P0,None,9495,4.102,5.052,5.067,NaN,SUB572_None_9495
2,SUB572,1.580992e+09,572.0,P0,None,7021,4.997,3.344,4.033,NaN,SUB572_None_7021
3,SUB572,1.580992e+09,572.0,PS-,288,3530,5.536,4.737,5.172,NaN,SUB572_288_3530
4,SUB572,1.580992e+09,572.0,P-,None,6200,4.471,4.504,3.883,NaN,SUB572_None_6200
...,...,...,...,...,...,...,...,...,...,...,...
11210,SUB915,1.580743e+09,915.0,PS0,701,7504,7.412,3.009,5.350,NaN,SUB915_701_7504
11211,SUB915,1.580743e+09,915.0,P0,None,9373,1.432,4.983,5.667,NaN,SUB915_None_9373
11212,SUB915,1.580743e+09,915.0,PS0,252,1820,2.988,3.313,2.667,NaN,SUB915_252_1820
11213,SUB915,1.580743e+09,915.0,P+,None,5460,9.000,8.501,5.533,NaN,SUB915_None_5460


### Load dataset with personalities of subjects

In [4]:
path='data-all/data-metadata.csv'
df_personality = pd.read_csv(path,delimiter=';')
cols_to_keep=['ID','OPENNESS','CONSCIENTIOUSNESS','EXTRAVERSION','AGREEABLENESS','NEUROTICISM']
df_personality=df_personality.drop(df_personality.columns.difference(cols_to_keep), axis=1)
df_personality.rename({'ID': 'SUB-NAME'}, axis=1, inplace=True)
df_personality['SUB-NAME'] = df_personality['SUB-NAME'].apply(lambda x: 'SUB{}'.format(x))

In [5]:
df_personality

,SUB-NAME,OPENNESS,CONSCIENTIOUSNESS,NEUROTICISM,AGREEABLENESS,EXTRAVERSION
0,SUB103,6,6,2,3,4
1,SUB119,6,7,6,2,2
2,SUB120,1,2,2,2,2
3,SUB136,7,3,8,5,10
4,SUB141,3,1,7,5,4
...,...,...,...,...,...,...
98,SUB948,7,10,1,3,7
99,SUB959,8,7,2,8,4
100,SUB969,3,8,6,1,5
101,SUB977,6,7,2,7,8


### load 'photo-subjects' dataset

In [6]:
path='data-all/data-face-after-crop'
subjects = os.listdir(path)
subjects =[x for x in subjects if x.endswith(".csv")]

df_all_subs=pd.DataFrame()

for sub in subjects:
    sub_name=sub.split('.csv')[0]
    df = pd.read_csv("{}/{}.csv".format(path,sub_name),delimiter=';')
    
    sub_name=sub_name.split('-')[0]
    df['SUB-NAME'] = df.apply(lambda row: sub_name, axis=1)
    df = df[['SUB-NAME','GAME-TIMESTAMP','FRAME-NUMBER', 'IADS-ID', 'IAPS-ID', 'ANGER','CONTEMPT','DISGUST','FEAR','HAPPINESS','NEUTRAL','SADNESS','SURPRISE']]
    frames=[df_all_subs,df]
    df_all_subs=pd.concat(frames)
    
def is_recognized(row):
    if math.isnan(row[5]):
        return 0
    else:
        return 1
    
#exclude rows where emotions not recognized.    
df_all_subs['FACE_RECOGNIZED'] = df_all_subs.apply(lambda row: is_recognized(row), axis=1) 

grouped_after=df_all_subs.groupby('FACE_RECOGNIZED')
not_recognized_df=grouped_after.get_group(0)
recognized_df=grouped_after.get_group(1)

recognized_df=recognized_df.reset_index()

recognized_df['STIMULI'] = recognized_df.apply(lambda row:'_'.join([str(recognized_df.loc[row.name,'SUB-NAME']),str(recognized_df.loc[row.name,'IADS-ID']),str(recognized_df.loc[row.name,'IAPS-ID'])]) , axis=1)

#exclude game stimuli
affective_df= recognized_df[~recognized_df['GAME-TIMESTAMP'].notna()]
del affective_df['index']

In [7]:
affective_df

,SUB-NAME,GAME-TIMESTAMP,FRAME-NUMBER,IADS-ID,IAPS-ID,ANGER,CONTEMPT,DISGUST,FEAR,HAPPINESS,NEUTRAL,SADNESS,SURPRISE,FACE_RECOGNIZED,STIMULI
0,SUB391,NaN,0,104,None,0.000,0.004,0.0,0.0,0.0,0.994,0.001,0.0,1,SUB391_104_None
1,SUB391,NaN,105,104,None,0.000,0.009,0.0,0.0,0.0,0.990,0.000,0.0,1,SUB391_104_None
2,SUB391,NaN,120,104,None,0.001,0.002,0.0,0.0,0.0,0.997,0.000,0.0,1,SUB391_104_None
3,SUB391,NaN,135,104,None,0.000,0.008,0.0,0.0,0.0,0.991,0.001,0.0,1,SUB391_104_None
4,SUB391,NaN,15,104,None,0.000,0.002,0.0,0.0,0.0,0.997,0.001,0.0,1,SUB391_104_None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528680,SUB410,NaN,45,None,9422,0.001,0.000,0.0,0.0,0.0,0.966,0.032,0.0,1,SUB410_None_9422
528681,SUB410,NaN,60,None,9422,0.004,0.001,0.0,0.0,0.0,0.909,0.087,0.0,1,SUB410_None_9422
528682,SUB410,NaN,75,None,9422,0.001,0.000,0.0,0.0,0.0,0.978,0.021,0.0,1,SUB410_None_9422
528683,SUB410,NaN,90,None,9422,0.001,0.000,0.0,0.0,0.0,0.972,0.027,0.0,1,SUB410_None_9422


### Exclude rows where no stimuli was presented (instructions, tutorial etc.)

In [8]:
affective_stimuli_lst = df_procedure[['STIMULI']]
affective_stimuli_lst = [ x[-1] for x in df_procedure.values.tolist()]
affective_df=affective_df[affective_df['STIMULI'].isin(affective_stimuli_lst)]

### df with baseline for each stimuli. Also, cut prestim from main df

In [9]:
#look up prestim to use it as a baseline for stat distances.
baseline_df=affective_df[affective_df['FRAME-NUMBER']=='prestim']
#delete 'prestim' (baseline) rows so stat distances are not 0.
affective_df = affective_df[affective_df['FRAME-NUMBER'] != 'prestim']

In [10]:
affective_df

,SUB-NAME,GAME-TIMESTAMP,FRAME-NUMBER,IADS-ID,IAPS-ID,ANGER,CONTEMPT,DISGUST,FEAR,HAPPINESS,NEUTRAL,SADNESS,SURPRISE,FACE_RECOGNIZED,STIMULI
0,SUB391,NaN,0,104,None,0.000,0.004,0.0,0.0,0.0,0.994,0.001,0.0,1,SUB391_104_None
1,SUB391,NaN,105,104,None,0.000,0.009,0.0,0.0,0.0,0.990,0.000,0.0,1,SUB391_104_None
2,SUB391,NaN,120,104,None,0.001,0.002,0.0,0.0,0.0,0.997,0.000,0.0,1,SUB391_104_None
3,SUB391,NaN,135,104,None,0.000,0.008,0.0,0.0,0.0,0.991,0.001,0.0,1,SUB391_104_None
4,SUB391,NaN,15,104,None,0.000,0.002,0.0,0.0,0.0,0.997,0.001,0.0,1,SUB391_104_None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528679,SUB410,NaN,345,None,9422,0.002,0.000,0.0,0.0,0.0,0.983,0.015,0.0,1,SUB410_None_9422
528680,SUB410,NaN,45,None,9422,0.001,0.000,0.0,0.0,0.0,0.966,0.032,0.0,1,SUB410_None_9422
528681,SUB410,NaN,60,None,9422,0.004,0.001,0.0,0.0,0.0,0.909,0.087,0.0,1,SUB410_None_9422
528682,SUB410,NaN,75,None,9422,0.001,0.000,0.0,0.0,0.0,0.978,0.021,0.0,1,SUB410_None_9422


### Add columns with stat distances

In [ ]:
#stat distances to apply:


def hellinger(p, q):
    return math.sqrt(sum([ (math.sqrt(p_i) - math.sqrt(q_i))**2 for p_i, q_i in zip(p, q) ]) / 2)

def bhattacharyya_distance(repr1, repr2):
#Calculates Bhattacharyya distance (https://en.wikipedia.org/wiki/Bhattacharyya_distance)."""
    sim = - np.log(np.sum([np.sqrt(p*q) for (p, q) in zip(repr1, repr2)]))
    assert not np.isnan(sim), 'Error: Similarity is nan.'
    if np.isinf(sim):
        # the similarity is -inf if no term in the review is in the vocabulary
        return 0
    return sim 

def get_entropy(f, s, mini=0.00000000001):
    # but entropy is not a metric!
    # replacing

    f[f == 0] = mini
    s[s == 0] = mini
    return st.entropy(f, s)


statistical_distances = ["mabse", "msqrte", "mse", "energy", "wasserstein",
                        "jensenshannon", "correlation",  ]



#in  case we theres no baseline for given stimulus, we can assume that baseline was 100% neutral expression. (???)
baseline_lst=[0,0,0,0,0,1.0,0,0]
functions=[(mt.mean_absolute_error,'mean_absolute_error'),(mt.mean_squared_error,'mean_squared_error'),
           (st.energy_distance,'energy'),(st.wasserstein_distance,'wasserstein'),
           (get_entropy,'relative entropy'),(sd.jensenshannon,'jensenshannon'),
           (hellinger,'hellinger'),(bhattacharyya_distance,'bhattacharyya_distance'),
           (sd.correlation,'correlation')
          ]

for func in functions:
    try:
        affective_df[func[1]] = affective_df.apply(lambda row: func[0](
                 row[['ANGER','CONTEMPT','DISGUST','FEAR','HAPPINESS','NEUTRAL','SADNESS','SURPRISE']].values.tolist(),
                 baseline_df.loc[baseline_df['STIMULI'] == row['STIMULI']][['ANGER','CONTEMPT','DISGUST','FEAR','HAPPINESS','NEUTRAL','SADNESS','SURPRISE']].values.tolist()[0]),
                 axis=1)
    except:
        affective_df[func[1]] = affective_df.apply(lambda row: func[0](
             row[['ANGER','CONTEMPT','DISGUST','FEAR','HAPPINESS','NEUTRAL','SADNESS','SURPRISE']].values.tolist(),
             baseline_lst),
             axis=1)

In [18]:
#affective_df.dropna(axis=0,inplace=False)

In [19]:
pd.set_option('display.max_columns', None)
affective_df

,SUB-NAME,GAME-TIMESTAMP,FRAME-NUMBER,IADS-ID,IAPS-ID,ANGER,CONTEMPT,DISGUST,FEAR,HAPPINESS,NEUTRAL,SADNESS,SURPRISE,FACE_RECOGNIZED,STIMULI,mean_absolute_error,mean_squared_error,energy,wasserstein,relative entropy,jensenshannon,hellinger,bhattacharyya_distance,correlation,1_highest emotion,2_highest emotion
0,SUB391,NaN,0,104,None,0.000,0.004,0.0,0.0,0.0,0.994,0.001,0.0,1,SUB391_104_None,0.001375,0.000007,0.020156,0.001375,inf,0.041686,0.050045,0.003009,0.000008,[NEUTRAL],[CONTEMPT]
1,SUB391,NaN,105,104,None,0.000,0.009,0.0,0.0,0.0,0.990,0.000,0.0,1,SUB391_104_None,0.002375,0.000023,0.024367,0.002375,inf,0.055969,0.067176,0.005025,0.000041,[NEUTRAL],[CONTEMPT]
2,SUB391,NaN,120,104,None,0.001,0.002,0.0,0.0,0.0,0.997,0.000,0.0,1,SUB391_104_None,0.000750,0.000002,0.015811,0.000750,inf,0.032262,0.038743,0.001502,0.000002,[NEUTRAL],[CONTEMPT]
3,SUB391,NaN,135,104,None,0.000,0.008,0.0,0.0,0.0,0.991,0.001,0.0,1,SUB391_104_None,0.002250,0.000018,0.025000,0.002250,inf,0.055940,0.067158,0.004520,0.000031,[NEUTRAL],[CONTEMPT]
4,SUB391,NaN,15,104,None,0.000,0.002,0.0,0.0,0.0,0.997,0.001,0.0,1,SUB391_104_None,0.000750,0.000002,0.015811,0.000750,inf,0.032262,0.038744,0.001502,0.000002,[NEUTRAL],[CONTEMPT]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528679,SUB410,NaN,345,None,9422,0.002,0.000,0.0,0.0,0.0,0.983,0.015,0.0,1,SUB410_None_9422,0.004250,0.000065,0.034460,0.004250,inf,0.076995,0.092392,0.008573,0.000112,[NEUTRAL],[SADNESS]
528680,SUB410,NaN,45,None,9422,0.001,0.000,0.0,0.0,0.0,0.966,0.032,0.0,1,SUB410_None_9422,0.008375,0.000273,0.046435,0.008375,inf,0.107643,0.129023,0.017296,0.000537,[NEUTRAL],[SADNESS]
528681,SUB410,NaN,60,None,9422,0.004,0.001,0.0,0.0,0.0,0.909,0.087,0.0,1,SUB410_None_9422,0.022875,0.001983,0.078062,0.022875,inf,0.181549,0.216991,0.047705,0.004510,[NEUTRAL],[SADNESS]
528682,SUB410,NaN,75,None,9422,0.001,0.000,0.0,0.0,0.0,0.978,0.021,0.0,1,SUB410_None_9422,0.005500,0.000116,0.037914,0.005500,inf,0.087669,0.105172,0.011123,0.000224,[NEUTRAL],[SADNESS]


### Add columns with first and second most recognized emotion

In [20]:
def emotion_val(row,top_no):
    #dict_emotions={0:'ANGER', 1:'CONTEMPT', 2:'DISGUST', 3:'FEAR', 4:'HAPPINESS', 5:'NEUTRAL', 6:'SADNESS', 7:'SURPRISE'}
    emotions=('ANGER', 'CONTEMPT', 'DISGUST', 'FEAR', 'HAPPINESS', 'NEUTRAL', 'SADNESS', 'SURPRISE')
    row=row.values.tolist()[5:12]
    row=zip(emotions,row)
    emotion_sorted=sorted(row,key=lambda x: x[1],reverse=True)
    
    result_1 = []
    result_2 = []
    for x,y in emotion_sorted:
        if y == emotion_sorted[0][1]:
            result_1.append(x)
        if y == emotion_sorted[1][1]:
            result_2.append(x)
    
    if emotion_sorted[top_no-1] == 0.0:
        return ['NONE']
    if top_no==1:
        return result_1
    if top_no==2:
        return result_2

affective_df['1_highest emotion'] = affective_df.apply(lambda row: emotion_val(row,1), axis=1)
affective_df['2_highest emotion'] = affective_df.apply(lambda row: emotion_val(row,2), axis=1)

In [21]:
full_df = affective_df.merge(df_procedure[['STIMULI',"COND", "ANS-VALENCE", "ANS-AROUSAL"]], on="STIMULI")

In [22]:
df_personality

,SUB-NAME,OPENNESS,CONSCIENTIOUSNESS,NEUROTICISM,AGREEABLENESS,EXTRAVERSION
0,SUB103,6,6,2,3,4
1,SUB119,6,7,6,2,2
2,SUB120,1,2,2,2,2
3,SUB136,7,3,8,5,10
4,SUB141,3,1,7,5,4
...,...,...,...,...,...,...
98,SUB948,7,10,1,3,7
99,SUB959,8,7,2,8,4
100,SUB969,3,8,6,1,5
101,SUB977,6,7,2,7,8


In [ ]:
# full_df.rename({'SUB-NAME': 'SUB_NAME'},axis=1,inplace=True)|
full_df = full_df.merge(df_personality[['SUB_NAME','OPENNESS','CONSCIENTIOUSNESS','NEUROTICISM','AGREEABLENESS','EXTRAVERSION']], on="SUB_NAME")

In [26]:
full_df

,SUB_NAME,GAME-TIMESTAMP,FRAME-NUMBER,IADS-ID,IAPS-ID,ANGER,CONTEMPT,DISGUST,FEAR,HAPPINESS,NEUTRAL,SADNESS,SURPRISE,FACE_RECOGNIZED,STIMULI,mean_absolute_error,mean_squared_error,energy,wasserstein,relative entropy,jensenshannon,hellinger,bhattacharyya_distance,correlation,1_highest emotion,2_highest emotion,COND,ANS-VALENCE,ANS-AROUSAL
0,SUB391,NaN,0,104,None,0.000,0.004,0.0,0.0,0.0,0.994,0.001,0.0,1,SUB391_104_None,0.001375,0.000007,0.020156,0.001375,inf,0.041686,0.050045,0.003009,0.000008,[NEUTRAL],[CONTEMPT],S0,4.702,6.466
1,SUB391,NaN,105,104,None,0.000,0.009,0.0,0.0,0.0,0.990,0.000,0.0,1,SUB391_104_None,0.002375,0.000023,0.024367,0.002375,inf,0.055969,0.067176,0.005025,0.000041,[NEUTRAL],[CONTEMPT],S0,4.702,6.466
2,SUB391,NaN,120,104,None,0.001,0.002,0.0,0.0,0.0,0.997,0.000,0.0,1,SUB391_104_None,0.000750,0.000002,0.015811,0.000750,inf,0.032262,0.038743,0.001502,0.000002,[NEUTRAL],[CONTEMPT],S0,4.702,6.466
3,SUB391,NaN,135,104,None,0.000,0.008,0.0,0.0,0.0,0.991,0.001,0.0,1,SUB391_104_None,0.002250,0.000018,0.025000,0.002250,inf,0.055940,0.067158,0.004520,0.000031,[NEUTRAL],[CONTEMPT],S0,4.702,6.466
4,SUB391,NaN,15,104,None,0.000,0.002,0.0,0.0,0.0,0.997,0.001,0.0,1,SUB391_104_None,0.000750,0.000002,0.015811,0.000750,inf,0.032262,0.038744,0.001502,0.000002,[NEUTRAL],[CONTEMPT],S0,4.702,6.466
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
256975,SUB410,NaN,345,None,9422,0.002,0.000,0.0,0.0,0.0,0.983,0.015,0.0,1,SUB410_None_9422,0.004250,0.000065,0.034460,0.004250,inf,0.076995,0.092392,0.008573,0.000112,[NEUTRAL],[SADNESS],P0,1.000,5.000
256976,SUB410,NaN,45,None,9422,0.001,0.000,0.0,0.0,0.0,0.966,0.032,0.0,1,SUB410_None_9422,0.008375,0.000273,0.046435,0.008375,inf,0.107643,0.129023,0.017296,0.000537,[NEUTRAL],[SADNESS],P0,1.000,5.000
256977,SUB410,NaN,60,None,9422,0.004,0.001,0.0,0.0,0.0,0.909,0.087,0.0,1,SUB410_None_9422,0.022875,0.001983,0.078062,0.022875,inf,0.181549,0.216991,0.047705,0.004510,[NEUTRAL],[SADNESS],P0,1.000,5.000
256978,SUB410,NaN,75,None,9422,0.001,0.000,0.0,0.0,0.0,0.978,0.021,0.0,1,SUB410_None_9422,0.005500,0.000116,0.037914,0.005500,inf,0.087669,0.105172,0.011123,0.000224,[NEUTRAL],[SADNESS],P0,1.000,5.000


In [27]:
del full_df['GAME-TIMESTAMP']

In [28]:
del full_df['FACE_RECOGNIZED']

In [29]:
full_df.sort_values(by=['STIMULI', 'NEUTRAL'],inplace=True)

In [ ]:
full_df=full_df[['SUB_NAME','STIMULI','FRAME-NUMBER','IADS-ID','IAPS-ID','ANGER','CONTEMPT','DISGUST','FEAR','HAPPINESS','NEUTRAL','SADNESS','SURPRISE','1_highest emotion','2_highest emotion','mean_absolute_error','mean_squared_error','energy','wasserstein','relative entropy','jensenshannon','hellinger','bhattacharyya_distance','correlation','COND','ANS-VALENCE','ANS-AROUSAL','OPENNESS','CONSCIENTIOUSNESS','NEUROTICISM','AGREEABLENESS','EXTRAVERSION']]

In [31]:
full_df

,SUB_NAME,FRAME-NUMBER,IADS-ID,IAPS-ID,ANGER,CONTEMPT,DISGUST,FEAR,HAPPINESS,NEUTRAL,SADNESS,SURPRISE,STIMULI,mean_absolute_error,mean_squared_error,energy,wasserstein,relative entropy,jensenshannon,hellinger,bhattacharyya_distance,correlation,1_highest emotion,2_highest emotion,COND,ANS-VALENCE,ANS-AROUSAL
87340,SUB103,300,102,None,0.001,0.207,0.0,0.0,0.000,0.789,0.003,0.0,SUB103_102_None,0.052750,0.010922,0.116190,0.052750,inf,0.281714,0.334280,0.118494,3.436184e-02,[NEUTRAL],[CONTEMPT],S0,6.582,5.00
87332,SUB103,195,102,None,0.000,0.022,0.0,0.0,0.000,0.974,0.003,0.0,SUB103_102_None,0.006375,0.000146,0.042205,0.006375,inf,0.093554,0.112186,0.013172,2.448778e-04,[NEUTRAL],[CONTEMPT],S0,6.582,5.00
87334,SUB103,225,102,None,0.000,0.001,0.0,0.0,0.000,0.985,0.014,0.0,SUB103_102_None,0.003750,0.000053,0.031623,0.003750,inf,0.072298,0.086766,0.007557,9.750467e-05,[NEUTRAL],[SADNESS],S0,6.582,5.00
87326,SUB103,120,102,None,0.000,0.000,0.0,0.0,0.000,0.992,0.008,0.0,SUB103_102_None,0.002000,0.000016,0.022361,0.002000,inf,0.052732,0.063309,0.004016,3.192657e-05,[NEUTRAL],[SADNESS],S0,6.582,5.00
87333,SUB103,210,102,None,0.000,0.000,0.0,0.0,0.000,0.993,0.006,0.0,SUB103_102_None,0.001625,0.000011,0.020156,0.001625,inf,0.045673,0.054828,0.003512,1.791256e-05,[NEUTRAL],[SADNESS],S0,6.582,5.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147546,SUB997,45,None,9594,0.000,0.000,0.0,0.0,0.002,0.996,0.002,0.0,SUB997_None_9594,0.001000,0.000003,0.019365,0.001000,inf,0.037260,0.044744,0.002004,3.295350e-06,[NEUTRAL],"[HAPPINESS, SADNESS]",P0,5.052,6.08
147531,SUB997,150,None,9594,0.000,0.001,0.0,0.0,0.001,0.997,0.001,0.0,SUB997_None_9594,0.000750,0.000002,0.019365,0.000750,inf,0.032262,0.038744,0.001502,9.863423e-07,[NEUTRAL],"[CONTEMPT, HAPPINESS, SADNESS]",P0,5.052,6.08
147536,SUB997,225,None,9594,0.000,0.001,0.0,0.0,0.002,0.997,0.000,0.0,SUB997_None_9594,0.000750,0.000002,0.015811,0.000750,inf,0.032262,0.038744,0.001502,2.137071e-06,[NEUTRAL],[HAPPINESS],P0,5.052,6.08
147539,SUB997,270,None,9594,0.000,0.001,0.0,0.0,0.002,0.997,0.001,0.0,SUB997_None_9594,0.000875,0.000002,0.020156,0.000875,inf,0.037241,0.044734,0.001502,2.137684e-06,[NEUTRAL],[HAPPINESS],P0,5.052,6.08


In [32]:
full_df.rename({
    
    'FRAME-NUMBER':'FRAME_NUMBER',
    'IADS-ID':'IADS_ID',
    'IAPS-ID':'IAPS_ID',
    'mean_absolute_error':'MEAN_ABS_ERR',
    'mean_squared_error':'MEAN_SQ_ERR',
    'energy':'ENERGY',
    'wasserstein':'WASSERSTEIN',
    'relative entropy':'RELATIVE_ENTROPY',
    'jensenshannon':'JENSENSHANNON',
    'hellinger':'HELLINGER',
    'bhattacharyya_distance':'BHATTACHARYYA_DIST',
    'correlation':'CORRELATION',
    '1_highest emotion':'1_HIGHEST_EMO',
    '2_highest emotion':'2_HIGHEST_EMO',
    'ANS-VALENCE':'ANS_VALENCE',
    'ANS-AROUSAL':'ANS_AROUSAL'
    
    }, axis=1, inplace=True)


In [33]:
full_df.rename({
    
    '1_HIGHEST_EMO':'FIRST_EMO',
    '2_HIGHEST_EMO':'SEC_EMO'

    
    }, axis=1, inplace=True)


In [34]:
full_df

,SUB_NAME,FRAME_NUMBER,IADS_ID,IAPS_ID,ANGER,CONTEMPT,DISGUST,FEAR,HAPPINESS,NEUTRAL,SADNESS,SURPRISE,STIMULI,MEAN_ABS_ERR,MEAN_SQ_ERR,ENERGY,WASSERSTEIN,RELATIVE_ENTROPY,JENSENSHANNON,HELLINGER,BHATTACHARYYA_DIST,CORRELATION,FIRST_EMO,SEC_EMO,COND,ANS_VALENCE,ANS_AROUSAL
87340,SUB103,300,102,None,0.001,0.207,0.0,0.0,0.000,0.789,0.003,0.0,SUB103_102_None,0.052750,0.010922,0.116190,0.052750,inf,0.281714,0.334280,0.118494,3.436184e-02,[NEUTRAL],[CONTEMPT],S0,6.582,5.00
87332,SUB103,195,102,None,0.000,0.022,0.0,0.0,0.000,0.974,0.003,0.0,SUB103_102_None,0.006375,0.000146,0.042205,0.006375,inf,0.093554,0.112186,0.013172,2.448778e-04,[NEUTRAL],[CONTEMPT],S0,6.582,5.00
87334,SUB103,225,102,None,0.000,0.001,0.0,0.0,0.000,0.985,0.014,0.0,SUB103_102_None,0.003750,0.000053,0.031623,0.003750,inf,0.072298,0.086766,0.007557,9.750467e-05,[NEUTRAL],[SADNESS],S0,6.582,5.00
87326,SUB103,120,102,None,0.000,0.000,0.0,0.0,0.000,0.992,0.008,0.0,SUB103_102_None,0.002000,0.000016,0.022361,0.002000,inf,0.052732,0.063309,0.004016,3.192657e-05,[NEUTRAL],[SADNESS],S0,6.582,5.00
87333,SUB103,210,102,None,0.000,0.000,0.0,0.0,0.000,0.993,0.006,0.0,SUB103_102_None,0.001625,0.000011,0.020156,0.001625,inf,0.045673,0.054828,0.003512,1.791256e-05,[NEUTRAL],[SADNESS],S0,6.582,5.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
147546,SUB997,45,None,9594,0.000,0.000,0.0,0.0,0.002,0.996,0.002,0.0,SUB997_None_9594,0.001000,0.000003,0.019365,0.001000,inf,0.037260,0.044744,0.002004,3.295350e-06,[NEUTRAL],"[HAPPINESS, SADNESS]",P0,5.052,6.08
147531,SUB997,150,None,9594,0.000,0.001,0.0,0.0,0.001,0.997,0.001,0.0,SUB997_None_9594,0.000750,0.000002,0.019365,0.000750,inf,0.032262,0.038744,0.001502,9.863423e-07,[NEUTRAL],"[CONTEMPT, HAPPINESS, SADNESS]",P0,5.052,6.08
147536,SUB997,225,None,9594,0.000,0.001,0.0,0.0,0.002,0.997,0.000,0.0,SUB997_None_9594,0.000750,0.000002,0.015811,0.000750,inf,0.032262,0.038744,0.001502,2.137071e-06,[NEUTRAL],[HAPPINESS],P0,5.052,6.08
147539,SUB997,270,None,9594,0.000,0.001,0.0,0.0,0.002,0.997,0.001,0.0,SUB997_None_9594,0.000875,0.000002,0.020156,0.000875,inf,0.037241,0.044734,0.001502,2.137684e-06,[NEUTRAL],[HAPPINESS],P0,5.052,6.08


In [38]:
full_df.to_csv('data-all/emo_personality_procedure_df.csv', index = False)

#... move on to pics_personality_classify.ipynb